In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import time
from tqdm import tqdm

%matplotlib inline

#每次可以输出多个变量
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from pylab import rcParams
rcParams['figure.figsize'] = 14, 6

#中文字体
import matplotlib  
matplotlib.use('qt4agg')  
#指定默认字体  
matplotlib.rcParams['font.sans-serif'] = ['SimHei']   
matplotlib.rcParams['font.family']='sans-serif'  
#解决负号'-'显示为方块的问题  
matplotlib.rcParams['axes.unicode_minus'] = False 

/usr/local/miniconda2/envs/py35/lib/python3.5/site-packages/matplotlib/__init__.py:1405: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [7]:
def mall_wifi_dateframe(mall_id, path):
    cols = pickle.load(open('../cjy/input/'+path+'/'+mall_id+'_cols.p','rb'))
    coo = pickle.load(open('../cjy/input/'+path+'/'+mall_id+'.p','rb'))
    coo = coo.tocoo(copy=False)
    coo_group = pd.DataFrame({'row': coo.row, 'col': coo.col, 'data': coo.data}
                 )[['row', 'col', 'data']].sort_values(['row', 'col']
                 ).reset_index(drop=True).groupby('col')
    df = pd.DataFrame(0 * np.ones((max(coo.row)+1, len(cols))), columns=cols, dtype=np.int64)
    tmp = coo_group.apply(lambda x:x.row.values)
    for col,rows,values in zip(tmp.index.tolist(),tmp.values,
                                coo_group.apply(lambda x:x.data.tolist())):
        df.iloc[rows,col]=values
    return df

In [8]:
train_shop = pd.read_csv('../input/训练数据-ccf_first_round_shop_info.csv')
train_user = pd.read_csv('../input/训练数据-ccf_first_round_user_shop_behavior.csv')
test_shop = pd.read_csv('../input/AB榜测试集-evaluation_public.csv')

train_shop.shape
train_shop.head()
train_user.shape
train_user.head()
test_shop.shape
test_shop.head()

(8477, 6)

,shop_id,category_id,longitude,latitude,price,mall_id
0,s_26,c_4,122.346736,31.833507,57,m_690
1,s_133,c_6,121.134362,31.197511,58,m_6587
2,s_251,c_38,121.000505,30.907667,34,m_5892
3,s_372,c_30,119.864982,26.659876,44,m_625
4,s_456,c_26,122.594243,31.581499,44,m_3839


(1138015, 6)

,user_id,shop_id,time_stamp,longitude,latitude,wifi_infos
0,u_376,s_2871718,2017-08-06 21:20,122.308291,32.088040,b_6396480|-67|false;b_41124514|-86|false;b_287...
1,u_376,s_2871718,2017-08-06 21:20,122.308162,32.087970,b_6396480|-67|false;b_56328155|-73|false;b_411...
2,u_1041,s_181637,2017-08-02 13:10,117.365255,40.638214,b_8006367|-78|false;b_2485110|-52|false;b_3005...
3,u_1158,s_609470,2017-08-13 12:30,121.134451,31.197416,b_26250579|-73|false;b_26250580|-64|false;b_26...
4,u_1654,s_3816766,2017-08-25 19:50,122.255867,31.351320,b_39004150|-66|false;b_39004148|-58|false;b_21...


(483931, 7)

,row_id,user_id,mall_id,time_stamp,longitude,latitude,wifi_infos
0,118742,u_30097142,m_3916,2017-09-05 13:00,122.141011,39.818847,b_34366982|-82|false;b_37756289|-53|false;b_41...
1,118743,u_30097803,m_5085,2017-09-06 13:10,118.191907,32.855858,b_36722251|-81|false;b_10537579|-75|false;b_43...
2,118744,u_30097889,m_4033,2017-09-06 17:40,119.192110,32.424667,b_30026291|-74|false;b_30026290|-74|false;b_36...
3,118745,u_30098996,m_4515,2017-09-03 12:10,120.612201,34.055249,b_33412374|-77|false;b_22084893|-86|false;b_52...
4,118746,u_30099170,m_7168,2017-09-02 20:40,116.861989,40.326858,b_19882704|-77|false;b_2241462|-49|false;b_585...


In [9]:
train_mall_ids = set(train_shop.mall_id)
test_mall_ids = set(test_shop.mall_id)
userShop = train_user.shop_id
userShop.head()

del train_shop
del train_user
del test_shop

0    s_2871718
1    s_2871718
2     s_181637
3     s_609470
4    s_3816766
Name: shop_id, dtype: object

In [10]:
from tqdm import tqdm
from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import cosine

In [4]:
import os  
  
finish_mall = os.listdir('./output/train_user_neg/')
for i in range(len(finish_mall)):
    finish_mall[i] = finish_mall[i][:-2]
finish_mall

['m_4548',
 'm_6803',
 'm_1089',
 'm_4422',
 'm_5529',
 'm_3528',
 'm_3445',
 'm_4033',
 'm_3839',
 'm_3313',
 'm_2333',
 'm_9068',
 'm_3517',
 'm_5810',
 'm_9054',
 'm_5154',
 'm_979',
 'm_1950',
 'm_2224',
 'm_5085',
 'm_2467',
 'm_4923',
 'm_7374',
 'm_626',
 'm_7168',
 'm_4459',
 'm_8379',
 'm_6337',
 'm_2058',
 'm_622',
 'm_4011',
 'm_2415',
 'm_1621',
 'm_2878',
 'm_1790',
 'm_1920',
 'm_3005',
 'm_3501',
 'm_615',
 'm_6587',
 'm_3832',
 'm_7601',
 'm_3425',
 'm_1085',
 'm_2267',
 'm_1021',
 'm_5767',
 'm_4341',
 'm_8344',
 'm_909',
 'm_1175',
 'm_625',
 'm_2123',
 'm_822',
 'm_4168',
 'm_2907',
 'm_7800',
 'm_2182',
 'm_690',
 'm_5892',
 'm_1409',
 'm_2715',
 'm_1293',
 'm_8093',
 'm_4543',
 'm_1375',
 'm_4406',
 'm_4094',
 'm_3871',
 'm_2270',
 'm_3739',
 'm_1831',
 'm_6167',
 'm_5352',
 'm_1263',
 'm_4515',
 'm_4572',
 'm_2578',
 'm_5076',
 'm_5825']

In [12]:
len(finish_mall)

97

In [ ]:
for mall_id in tqdm(train_mall_ids):
    if mall_id not in finish_mall:
        mall = mall_wifi_dateframe(mall_id, 'train_matrix')
        temp = pd.DataFrame(1-pairwise_distances(mall.drop(['full_order'], axis=1).as_matrix(), metric="cosine"))
        full_order = mall.full_order
        user_frame = pd.DataFrame()
    
        for i in range(temp.shape[0]):
            k = 8
            p = 0
            q = 0
            user = []
            shop_list = []
            user_raw = full_order[i]
            shop_id = userShop[user_raw]
            shop_list.append(shop_id)
            user.append(user_raw)
            index = temp.iloc[i].sort_values(ascending=False).index
            while p < k:
                if (userShop[full_order[index[q]]] not in shop_list) & (full_order[index[q]] != user_raw):
                    user.append(full_order[index[q]])
                    shop_list.append(userShop[full_order[index[q]]])
                    q = q + 1
                    p = p + 1
                else:
                    q = q + 1
            user_frame = pd.concat([user_frame, pd.DataFrame(user)], axis=1)
        
        print(mall_id)
        finish_mall.append(mall_id)
        pickle.dump(user_frame, open('./output/train_user_neg/' + str(mall_id) + '.p', 'wb'), protocol=4)

  0%|          | 0/97 [00:00<?, ?it/s]